In [44]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [4]:
train_df = pd.read_csv("D:\Python_files\weight_prediction\Titantic-train.csv")
test_df = pd.read_csv("D:\Python_files\weight_prediction\Titantic-test.csv")

train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
train_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [11]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [9]:
train_df.duplicated().sum()

0

In [10]:
train_df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [12]:
test_df.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

Process Data

In [35]:
train = train_df.drop(['PassengerId', 'Cabin', 'Ticket', 'Name', 'Survived'], axis=1)
test = test_df.drop(['PassengerId', 'Cabin', 'Ticket', 'Name'], axis=1)

In [36]:
train['Embarked'] = train['Embarked'].fillna(train['Embarked'].mode()[0]) # most frequent value

In [37]:
#train['Age'] = train['Age'].fillna(train['Age'].median())
train['Age'] = train.groupby(['Sex', 'Pclass'])['Age'].transform(lambda x: x.fillna(x.median()))
test['Age'] = test.groupby(['Sex', 'Pclass'])['Age'].transform(lambda x: x.fillna(x.median()))

In [40]:
le = LabelEncoder()
train.Embarked = le.fit_transform(train.Embarked)
test.Embarked = le.transform(test.Embarked)

In [41]:
train['Sex'] = train['Sex'].map({'male': 1, 'female': 0})
test['Sex'] = test['Sex'].map({'male': 1, 'female': 0})

In [42]:
cols = ['Age', 'Fare']
scaler = StandardScaler()
train[cols] = scaler.fit_transform(train[cols])
test[cols] = scaler.transform(test[cols])
# scaler = StandardScaler()
# train['Age'] = scaler.fit_transform(train['Age']) 
# test['Age'] = scaler.transform(test['Age'])
# scaler = StandardScaler()
# train['Fare'] = scaler.fit_transform(train['Fare'])
# test['Fare'] = scaler.transform(test['Fare'])

In [43]:
train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,-0.534891,1,0,-0.502445,2
1,1,0,0.668392,1,0,0.786845,0
2,3,0,-0.234070,0,0,-0.488854,2
3,1,0,0.442776,1,0,0.420730,2
4,3,1,0.442776,0,0,-0.486337,2


In [45]:
X = train
y = train_df['Survived']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [46]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

randomforest = RandomForestClassifier()

# Fit the training data along with its output
randomforest.fit(x_train, y_train)
y_pred = randomforest.predict(x_test)

# Find the accuracy score of the model
acc_randomforest = round(accuracy_score(y_pred, y_test) * 100, 2)
print(acc_randomforest)

82.68


In [53]:
ids = test_df['PassengerId']
predictions = randomforest.predict(test)

# set the output as a dataframe and convert 
# to csv file named resultfile.csv
output = pd.DataFrame({'PassengerId': ids, 'Predicted': predictions})
#output.to_csv('resultfile.csv', index=False)
output

,PassengerId,Predicted
0,892,0
1,893,0
2,894,0
3,895,1
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.86      0.85       105
           1       0.79      0.78      0.79        74

    accuracy                           0.83       179
   macro avg       0.82      0.82      0.82       179
weighted avg       0.83      0.83      0.83       179



Precision: Of all samples predicted as a certain class, the proportion that are actually correct.  
Precision = True Positives / (True Positives + False Positives)

recall: Of all actual samples of a certain class, the proportion that are correctly identified.  
Recall = True Positives / (True Positives + False Negatives)

f1-score: The harmonic mean of precision and recall; it balances both metrics.  
F1-score = 2 × (Precision × Recall) / (Precision + Recall)

accuracy = number of correct predictions / total number of samples

In [56]:
model = tf.keras.Sequential([
      tf.keras.layers.Dense(32, activation = 'relu', input_shape = (7,)),
      tf.keras.layers.Dense(16, activation = 'relu'),
      tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs = 100)

Epoch 1/100
23/23 [==============================] - 2s 4ms/step - loss: 0.6686 - accuracy: 0.6348
Epoch 2/100
23/23 [==============================] - 0s 4ms/step - loss: 0.6226 - accuracy: 0.6503
Epoch 3/100
23/23 [==============================] - 0s 4ms/step - loss: 0.5928 - accuracy: 0.6643
Epoch 4/100
23/23 [==============================] - 0s 3ms/step - loss: 0.5688 - accuracy: 0.6854
Epoch 5/100
23/23 [==============================] - 0s 3ms/step - loss: 0.5464 - accuracy: 0.7121
Epoch 6/100
23/23 [==============================] - 0s 3ms/step - loss: 0.5276 - accuracy: 0.7486
Epoch 7/100
23/23 [==============================] - 0s 3ms/step - loss: 0.5118 - accuracy: 0.7612
Epoch 8/100
23/23 [==============================] - 0s 3ms/step - loss: 0.4968 - accuracy: 0.7584
Epoch 9/100
23/23 [==============================] - 0s 3ms/step - loss: 0.4841 - accuracy: 0.7851
Epoch 10/100
23/23 [==============================] - 0s 3ms/step - loss: 0.4735 - accuracy: 0.7949
Epoch 11/

In [ ]:
model.evaluate(x_test, y_test) #loss and accuracy

6/6 [==============================] - 0s 3ms/step - loss: 0.4233 - accuracy: 0.8212


[0.42334091663360596, 0.8212290406227112]

In [ ]:
y_pred = model.predict(x_test)
y_pred_binary = (y_pred > 0.5).astype(int) # 0.5 is the threshold for classification

6/6 [==============================] - 0s 1ms/step


In [69]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_binary))

              precision    recall  f1-score   support

           0       0.82      0.89      0.85       105
           1       0.82      0.73      0.77        74

    accuracy                           0.82       179
   macro avg       0.82      0.81      0.81       179
weighted avg       0.82      0.82      0.82       179



In [ ]:
model.predict(test) 

14/14 [==============================] - 0s 831us/step


array([[8.77879634e-02],
       [5.95328391e-01],
       [9.16431919e-02],
       [1.14396855e-01],
       [2.73577601e-01],
       [1.49268210e-01],
       [5.93636811e-01],
       [6.63131177e-02],
       [7.68820107e-01],
       [6.64996505e-02],
       [1.16154969e-01],
       [3.35995525e-01],
       [9.95202065e-01],
       [8.31191465e-02],
       [9.97976720e-01],
       [8.87079298e-01],
       [8.77753347e-02],
       [2.40484461e-01],
       [4.29368794e-01],
       [5.98389149e-01],
       [4.78412420e-01],
       [6.76026165e-01],
       [9.95588899e-01],
       [6.70030713e-01],
       [9.98886406e-01],
       [4.50855643e-02],
       [9.87642646e-01],
       [2.26556793e-01],
       [3.30073476e-01],
       [2.70084471e-01],
       [6.73710853e-02],
       [7.21402541e-02],
       [6.17487133e-01],
       [3.56492490e-01],
       [4.81788129e-01],
       [2.65088141e-01],
       [5.04190981e-01],
       [4.99965191e-01],
       [1.18770197e-01],
       [3.04423541e-01],
